<h1 style = 'color: orange'> Configurações </h1>

### Importação da Bibliotecas

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

import pandas as pd
import re
import datetime
import time
from sqlalchemy import create_engine

In [2]:
before = datetime.datetime.now()

### Conectar ao Banco de dados

In [ ]:
conexao = create_engine('postgresql://{user}:{password}@{host}:{port}/{database}'.format(
    host='db.yeqkmhmarjrfnxyocpnh.supabase.co', 
    port = 5432,
    database='postgres',
    user='postgres', 
    password='ThButewicz2409'
))

### Configuração do Selenium

In [ ]:
options = Options()
options.add_experimental_option("detach",True)

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),
                          options=options)

<h1 style = 'color: orange'> Juntar Pontos </h1>

#### OBter os dados

In [ ]:
# Abrir o Link da Livelo Juntar Pontos 
driver.get("https://www.livelo.com.br/ganhe-pontos-compre-e-pontue")
time.sleep(1.8)

In [ ]:
# Obter os CARDs de ofertas
cards = driver.find_elements(By.CLASS_NAME, 'parity__card')

In [ ]:
# Navegar pelos cards Obtendo as informações de cada Oferta
dados = []

for card in cards:
    
    # Obter as informações cruciais de cada CARD
    dado = {
        "nome":card.find_element(By.CLASS_NAME,'parity__card--img')\
                                .get_attribute('alt'),
        "pontos": card.find_element(By.CLASS_NAME,'parity__card--info')\
                                .text.replace('\n',' ').split('R$')[-1],
        "link" : card.find_element(By.CLASS_NAME,'gtm-link-event')\
                                .get_attribute('href')
    }
    
    dados.append(dado)
    
# Criar o DataFrame
df = pd.DataFrame(dados)

In [ ]:
df

,nome,pontos,link
0,ABC da Construcao,1 até 10 Pontos Livelo,https://www.livelo.com.br/ganhe-pontos-compre-...
1,Abelha Rainha,1 até 14 pontos no Clube Livelo ou até 7 Pont...,https://www.livelo.com.br/ganhe-pontos-compre-...
2,Aguativa Resort,1 = 2 Pontos Livelo,https://www.livelo.com.br/ganhe-pontos-compre-...
3,Aliexpress,U$ 1 até 3 pontos no Clube Livelo ou até 1 Pon...,https://www.livelo.com.br/ganhe-pontos-compre-...
4,Allianz,1 = 4 Pontos Livelo,https://www.livelo.com.br/ganhe-pontos-compre-...
...,...,...,...
256,Wish Hotels,1 = 8 Pontos Livelo,https://www.livelo.com.br/ganhe-pontos-compre-...
257,Wishin,1 = 2 Pontos Livelo,https://www.livelo.com.br/ganhe-pontos-compre-...
258,Yvy,1 = 3 Pontos Livelo,https://www.livelo.com.br/ganhe-pontos-compre-...
259,Zattini,1 = 3 Pontos Livelo,https://www.livelo.com.br/ganhe-pontos-compre-...


#### Manipular os dados

In [ ]:
# Regex para obter o palavra antes da ultimo "ponto" da String
pattern = r'(\d+)\s*Ponto[s]?\s+Livelo(?![\s\S]*\bPonto\s+Livelo)'

# Ajustar a coluna de Pontos
df['pontos'] = df['pontos'].apply(lambda string: re.search(pattern, string).group(1))
df['referencia'] = datetime.date.today()
df['pontos'] = df['pontos'].astype(int)

### Acessar cada campanha para pegar as datas

In [ ]:
df['inicio'] = None
df['fim'] = None

# RegEx para encontrar as datas que preciso.
pattern_begin = r'\b\d{2}\b'
pattern_end =  r'\d{2}/\d{2}/\d{4}'

In [ ]:
for index,link in enumerate(df['link']):
    
    # Skipar os casos onde são menos de 3 Pontos = Campanhas vitalicias
    if df['pontos'].iloc[index] <= 3: continue
    
    # Acessar a Campanha
    driver.get(link)
    time.sleep(1.9)
    
    # Obter a string que contém essas informações
    string_oferta = driver.find_element(By.CLASS_NAME,'banner__blocktext-text').text\
                                                                                .split('LIVELO')[-1]
    
    # Obter as datas
    try:
        data_final = datetime.datetime.strptime(re.search(pattern_end, string_oferta).group(),'%d/%m/%Y').date()

        df['inicio'].iloc[index] = datetime.date(data_final.year, data_final.month, \
                                                 int(re.search(pattern_begin, string_oferta).group()))
        df['fim'].iloc[index] = data_final
        
    except:
        continue

C:\Users\zThan\AppData\Local\Temp\ipykernel_12852\2513902795.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['inicio'].iloc[index] = datetime.date(data_final.year, data_final.month, \
C:\Users\zThan\AppData\Local\Temp\ipykernel_12852\2513902795.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['fim'].iloc[index] = data_final


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".banner__blocktext-text"}
  (Session info: chrome=116.0.5845.188)
Stacktrace:
	GetHandleVerifier [0x008037C3+48947]
	(No symbol) [0x00798551]
	(No symbol) [0x0069C92D]
	(No symbol) [0x006C9E38]
	(No symbol) [0x006C9EFB]
	(No symbol) [0x006F8EF2]
	(No symbol) [0x006E50D4]
	(No symbol) [0x006F75DA]
	(No symbol) [0x006E4E86]
	(No symbol) [0x006C16C7]
	(No symbol) [0x006C284D]
	GetHandleVerifier [0x00A4FDF9+2458985]
	GetHandleVerifier [0x00A9744F+2751423]
	GetHandleVerifier [0x00A91361+2726609]
	GetHandleVerifier [0x00880680+560624]
	(No symbol) [0x007A238C]
	(No symbol) [0x0079E268]
	(No symbol) [0x0079E392]
	(No symbol) [0x007910B7]
	BaseThreadInitThunk [0x76717BA9+25]
	RtlInitializeExceptionChain [0x779AB79B+107]
	RtlClearBits [0x779AB71F+191]


In [ ]:
# Filtrar a ordem das colunas
df = df[['referencia','nome','pontos','inicio','fim','link']]

## Verificar ofertas já inseridas

In [ ]:
# Obter as ofertas
ofertas = pd.read_sql('select * from livelo_ofertas', conexao)

# Adicionar ao dataframe
df = df.append(ofertas)

In [ ]:
# Excluir os dados duplicados
df = df.drop_duplicates(subset = ['nome','inicio','fim'])

In [ ]:
# Inserir os novos dados
df.to_sql('livelo_ofertas', conexao, if_exists='replace', index = False)

<h1 style = 'color: orange'> Transferências de Pontos </h1>

In [ ]:
driver.get('https://www.livelo.com.br/transfira-seus-pontos-para-outros-programas')

In [ ]:
# Obter os CARDs de ofertas
cards = driver.find_elements(By.CLASS_NAME, 'itemBox')

time.sleep(1.7)

In [ ]:
dados = [] 

for card in cards:
    dado = {
    'programa' : card.find_element(By.CLASS_NAME,'itemBox__image').find_element(By.TAG_NAME,'img').get_attribute('alt'),
    'pontos'   : card.find_element(By.CLASS_NAME,'itemBox__description').text.replace('\n',' ')
    }
    dados.append(dado)

transf = pd.DataFrame(dados)

In [ ]:
pattern_number = r'\b\d+(\.\d+)?\b'

transf['pontos_livelo'] = transf['pontos'].apply(lambda string: re.search(pattern_number, string.split('=')[0]).group())
transf['pontos_programa'] = transf['pontos'].apply(lambda string: re.search(pattern_number, string.split('=')[-1]).group())

transf = transf.drop(columns = ['pontos'])

### Inserir no Banco

In [ ]:
transf.to_sql('livelo_transferencias', conexao, if_exists = 'replace', index = False)

In [ ]:
conexao.dispose()

In [ ]:
after = datetime.datetime.now()
print(f'Tempo de Execução: {after - before}')